# Exercice 1 :

###  1)

In [2]:
%%writefile hello_world_mpi.py

from mpi4py import MPI


COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

print("Hello world !")
    

Overwriting hello_world_mpi.py


In [3]:
!mpirun -n 4 python3  hello_world_mpi.py 

Hello world !
Hello world !
Hello world !
Hello world !


### 2)

In [4]:
%%writefile hello_world_mpi_r_s.py

from mpi4py import MPI


COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

def h_w():
    print("Hello world !",RANK,SIZE)
    
h_w()

Overwriting hello_world_mpi_r_s.py


In [5]:
!mpirun --oversubscribe -n 8 python3  hello_world_mpi.py #to use more than the real cores existing on the machine

Hello world !
Hello world !
Hello world !
Hello world !
Hello world !
Hello world !
Hello world !
Hello world !


### 3)

In [6]:
%%writefile hello_world_mpi_r_s_0.py

from mpi4py import MPI


COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

if RANK==0 :
    print("Hello world !",RANK,SIZE)

Overwriting hello_world_mpi_r_s_0.py


In [7]:
!mpirun -n 4 python3  hello_world_mpi.py 

Hello world !
Hello world !
Hello world !
Hello world !


# Exercice 2 :


In [7]:
%%writefile neg_value_exit.py

from mpi4py import MPI

# Initialize MPI environment
COMM = MPI.COMM_WORLD

# Get the rank of the current process
RANK= COMM.Get_rank()

while True:
    if RANK == 0:
        # Only the controller process (rank 0) reads the input
        value = int(input("Enter an integer value (or a negative integer to exit):"))
        
    else:
        value = None
        
    # Send the value to all processes
    value = COMM.bcast(value, root=0)
    
    # Exit the program in case of negative input
    if value < 0:
        break

    
    # Print the received value and the rank of the current process
    print(f"Process {RANK} got {value}")
    COMM.Barrier()

In [8]:
!mpirun -n 4 python3  neg_value_exit.py

# Exercice 3 :

### 1)

In [16]:
%%writefile broadcast_by_ring.py

from mpi4py import MPI

# Initialize MPI environment
COMM = MPI.COMM_WORLD

# Get the rank of the current process
RANK= COMM.Get_rank()

# Get the number of the machine cores
SIZE = COMM.Get_size()


while True:
    
    if RANK == 0:
        
        #Only the controller process (rank 0) reads the input
        value = int(input("Enter an integer value (or a negative integer to exit):"))
        
        COMM.send( value, dest= 1)
        
    
    else :
        
        value = COMM.recv( source= RANK-1)
        
        if RANK < SIZE-1 :
            COMM.send( value + RANK, dest= RANK + 1)
        
        
    if value < 0 :
        
        break
        
   
        
    # Print the received value and the rank of the current process
    
    print(f"Process {RANK} got {value}")
    COMM.Barrier()
        


        
        
        
        
        
        

# Exercice 4 :

In [20]:
import numpy as np
from mpi4py import MPI
COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

#Question 1 :

n, m = map(int, input().split())

# create the matrix on process 0
if RANK == 0:
    matrix = np.arange(n*m, dtype='float64').reshape(n, m)
    print(matrix)
else:
    matrix = None
    
#Question 2 :

# divide the matrix into the required submatrixes parts and scatter to the other processes

#submatrix of processor 0
sub_matrix_0 = matrix[ 0 : (n/2)-1 , 0 : (m/2)-1 ]

#submatrix of processor 1
sub_matrix_1 = matrix[ 0 : (n/2)-1 , m/2 : m-1 ]

#submatrix of processor 2
sub_matrix_2 = matrix[ n/2 : n-1 , 0 : (m/2)-1 ]

#submatrix of processor 3
sub_matrix_3 = matrix[ n/2 : n-1 , m/2 : m-1 ]

#constructing array of submatrix to help the scatter function

subm_arr = np.array([sub_matrix_0,sub_matrix_1,sub_matrix_2,sub_matrix_3])

data = COMM.scatterv(subm_arr, root=0)

''' 
# Scatter the matrix parts to each processor
recvA = np.zeros((n // 2, m // 2))
recvcounts = (n // 2) * (m // 2)
print(np.transpose(A))
comm.Scatterv([np.transpose(A) , sendcounts, displs, MPI.DOUBLE], recvA, root=0)
if rank == 1:
    print("Received matrix on processor 1:")
    print(recvA)
elif rank == 2:
    print("Received matrix on processor 2:")
    print(recvA)
elif rank == 3:
    print("Received matrix on processor 2:")
    print(recvA)
'''

4 4
[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]]


TypeError: slice indices must be integers or None or have an __index__ method

In [21]:
from mpi4py import MPI
import numpy as np

# Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Define matrix size
n = 6
m = 4

# Create matrix A on processor 0
if rank == 0:
    A = np.arange(n*m).reshape((n, m))
    print("Matrix A:\n", A)
else:
    A = None

# Define scatterv parameters
counts = [int(n*m/size)] * size
counts[0] = n*m - counts[1:]
displs = [sum(counts[:i]) for i in range(size)]

# Scatter parts of matrix A to other processors
recvbuf = np.zeros(int(n*m/size), dtype='i')
comm.Scatterv([A, counts, displs, MPI.INT], recvbuf, root=0)

# Processor 1 receives A(i,j) for i=0 to (n/2)-1 and j=m/2 to m-1
if rank == 1:
    recvbuf = recvbuf.reshape((int(n/2), int(m/2)))
    print("Processor", rank, "received:\n", recvbuf)

# Processor 2 receives A(i,j) for i=n/2 to n-1 and j=0 to (m/2)-1
if rank == 2:
    recvbuf = recvbuf.reshape((int(n/2), int(m/2)))
    print("Processor", rank, "received:\n", recvbuf)

# Processor 3 receives A(i,j) for i=n/2 to n-1 and j=m/2 to m-1
if rank == 3:
    recvbuf = recvbuf.reshape((int(n/2), int(m/2)))
    print("Processor", rank, "received:\n", recvbuf)


Matrix A:
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]
 [20 21 22 23]]


TypeError: unsupported operand type(s) for -: 'int' and 'list'

# Exercice 5 :

In [ ]:
import numpy as np
import scipy
from scipy.sparse import lil_matrix
from numpy.random import rand, seed
from numba import njit
from mpi4py import MPI


''' This program compute parallel csc matrix vector multiplication using mpi '''

COMM = MPI.COMM_WORLD
size = COMM.Get_size()
RANK = COMM.Get_rank()

seed(42)

def matrixVectorMult(A, b, x, size):
    
    row, col = A.shape
    start = RANK * (row // size)
    end = start + (row // size)
   #x = np.zeros(col)
    for i in range(start, end):
       # a = A[i]
        for j in range(col):
            x[i] += A[i,j] * b[j]

    return x

########################initialize matrix A and vector b ######################
#matrix sizes
SIZE = 1000
#Local_size = 

# counts = block of each proc
#counts = 

if RANK == 0:
    A = lil_matrix((SIZE, SIZE))
    A[0, :100] = rand(100)
    A[1, 100:200] = A[0, :100]
    A.setdiag(rand(SIZE))
    A = A.toarray()
    b = rand(SIZE)
else :
    A = None
    b = None


#########Send b to all procs and scatter A (each proc has its own local matrix#####
LocalMatrix = lil_matrix((SIZE, SIZE))
# Scatter the matrix A
#COMM.Scatter(A, LocalMatrix, root=0)
#####################Compute A*b locally#######################################
b = np.zeros(SIZE)
Localb = np.zeros(SIZE) 
#LocalX = np.zeros(SIZE)
#COMM.Scatter(b, Localb, root=0)

LocalX = np.zeros(SIZE)
LocalX = matrixVectorMult(LocalMatrix, b, LocalX, size)
if RANK == 0:
    X = np.zeros(SIZE)

#recvcounts = np.full(4, SIZE // 4)
#displs = np.arange(4) * (SIZE // 4)
#COMM.Gatherv(LocalX, [X, recvcounts, displs, MPI.DOUBLE], root = 0)
# start = MPI.Wtime()
# matrixVectorMult(LocalMatrix, b, LocalX)
# stop = MPI.Wtime()

if RANK == 0:
    start = MPI.Wtime()
    matrixVectorMult(LocalMatrix, b, LocalX, size)
    stop = MPI.Wtime()
    print("CPU time of parallel multiplication is ", (stop - start))

##################Gather te results ###########################################
# sendcouns = local size of result
#sendcounts = 
if RANK == 0:
    X = matrixVectorMult(A, b, LocalX, size) 
else :
    X = None

# Gather the result into X


##################Print the results ###########################################

if RANK == 0 :
    start = MPI.Wtime()
    X_ = A.dot(b)
    stop = MPI.Wtime()
    print("The time to calculate A*b using dot is :", stop - start)
    # print("The result of A*b using parallel version is :", X)
    
    

if RANK == 0:
    import matplotlib.pyplot as plt
    plt.plot(X)
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.title('Parallel Matrix-Vector Multiplication')
    plt.show()

# Exercice 6 :

In [ ]:


from mpi4py import MPI
import math

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

N = 840

pi_part = 0.0
for i in range(rank, N, size):
    pi_part += 1.0/(1.0 + ((i + 0.5)/N)**2)
pi_part *= 4.0/N
#Print the computed value of pi for each process
print("Process", rank, "partial sum:", pi_part)
if rank == 0:
    pi = pi_part
    for i in range(1, size):
        pi += comm.recv(source=i)
else:
    comm.send(pi_part, dest=0)
if rank == 0:
    print("Computed pi:", pi)
comm.Barrier() 
t1 = MPI.Wtime()
num_runs = 10000 
for i in range(num_runs):
    pi_part = 0.0
    for j in range(rank, N, size):
        #Divide the iterations among the processes
        pi_part += 1.0/(1.0 + ((j + 0.5)/N)**2)
    pi_part *= 4.0/N
    #Collect and sum the partial results on the rank 0 process
    if rank == 0:
        pi = pi_part
        for j in range(1, size):
            pi += comm.recv(source=j)
    else:
        comm.send(pi_part, dest=0)
    if rank == 0 and i == 0:
        print("Computed pi:", pi)
comm.Barrier()
t2 = MPI.Wtime()
if rank == 0:
    print("Taken Time:", t2 - t1)